# Arbitrary state preparation using Model-Free Reinforcement Learning

This notebook showcases an application of the formalism introduced in PhysRevX.12.011059 (https://doi.org/10.1103/PhysRevX.12.011059) on arbitrary qubit state preparation, as depicted in the Appendix D.2b.

The implementation of the quantum environment is done here via Qiskit, using an Estimator primitive (https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials/how-to-getting-started-with-estimator.html) for the execution of parametrized quantum circuits and Pauli expectation sampling.

Author of notebook: Arthur Strauss

Updated on 21/02/2024

In [1]:
import numpy as np
from rl_qoc import QuantumEnvironment
from rl_qoc.helpers.tf_utils import select_optimizer, generate_model
from rl_qoc.config import QEnvConfig, BackendConfig, ExecutionConfig
from rl_qoc.environment import StateTarget

# Qiskit imports for building RL environment (circuit level)
from qiskit.circuit import ParameterVector, QuantumCircuit, QuantumRegister
from qiskit.quantum_info import DensityMatrix

# Tensorflow imports for building RL agent and framework
import tensorflow as tf
from tensorflow_probability.python.distributions import MultivariateNormalDiag
from gymnasium.spaces import Box

# Additional imports
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
# Ansatz function, could be at pulse level or circuit level
from rl_qoc.circuits import apply_parametrized_circuit

# Defining the QuantumEnvironment

Below, we set the RL environment parameters, that is how we describe our quantum system. Below, we can choose to go through the use of Qiskit Runtime, or to speed things up by using the local CPU and a state-vector simulator to get measurement outcomes based on the ansatz circuit defined above. The Environment is defined as a class object called QuantumEnvironment.

In [3]:
qubit_tgt_register = [0, 1, 2]
sampling_Paulis = 50
batch_size = 200
n_shots = 1
n_actions = 14
seed = 3590
estimator_options = {"seed_simulator": seed, "resilience_level": 0}
action_space = Box(low=-1, high=1, shape=(n_actions,), dtype=np.float32)
observation_space = Box(low=-1, high=1, shape=(2,), dtype=np.float32)

Choose below which IBM Backend to use. As we are dealing with circuit level implementation, we can look for a backend supporting Qiskit Runtime (could be a cloud simulator, or real backend) or simply set backend to None and rely on the Estimator primitive based on statevector simulation. In either case, we need access to one Estimator primitive to run the algorithm, as the feedback from the measurement outcomes is done by calculating Pauli expectation values.

In [4]:
"""
Real backend initialization:
Run this cell only if intending to use a real backend,
where Qiskit Runtime is enabled
"""

backend_name = "ibm_perth"

# service = QiskitRuntimeService(channel='ibm_quantum')
# runtime_backend = service.get_backend(backend_name)
# estimator_options = {'resilience_level': 0}

In [5]:
"""
If using Qiskit native Estimator primitive
(statevector simulation)
"""

backend_config = BackendConfig(None, n_qubits=3, initial_layout=[0, 1, 2])

In [6]:
# Choose the backend_config to use for the training
backend_config = backend_config

In [7]:
# Define here target state density matrix

# Target state: GHZ state: (|000> + |111>)/sqrt(2)
ket0, ket1 = np.array([[1.0], [0]]), np.array([[0.0], [1.0]])
ket000, ket111 = np.kron(np.kron(ket0, ket0), ket0), np.kron(np.kron(ket1, ket1), ket1)
GHZ_state = (ket000 + ket111) / np.sqrt(2)
GHZ_dm = GHZ_state @ GHZ_state.conj().T
target_state = StateTarget(state=DensityMatrix(GHZ_dm), physical_qubits=[0, 1, 2])

In [8]:
# Wrap all info in one dict backend_config


In [9]:
# Declare QuantumEnvironment variable
q_env_config = QEnvConfig(
    target=target_state,
    backend_config=backend_config,
    action_space=action_space,
    execution_config=ExecutionConfig(
        batch_size=batch_size,
        sampling_paulis=sampling_Paulis,
        n_shots=n_shots,
    ),
    seed=seed,
)
q_env = QuantumEnvironment(q_env_config, apply_parametrized_circuit)
print(q_env.target)

StateTarget(DensityMatrix([[0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0.5+0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0. +0.j],
               [0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
                0. +0.j, 0.5+0.j]],
              dims=(2, 2, 2)) on qubits [0, 1, 2])


We now define the Agent, which will be in general a Deep Neural Network.
We start by defining the hyperparameters of the training

In [10]:
# Hyperparameters for the agent
n_epochs = 1500
opti = "Adam"
eta = 0.01
eta_2 = None

use_PPO = True
epsilon = 0.2
grad_clip = 0.1
critic_loss_coeff = 0.5
optimizer = select_optimizer(
    lr=eta, optimizer=opti, grad_clip=grad_clip, concurrent_optimization=True, lr2=eta_2
)
sigma_eps = 1e-3

In [11]:
# Policy parameters: generate NN that will output mean and variances of the policy

N_in = observation_space.shape[
    -1
]
hidden_units = [100, 100, 100]

network = generate_model((N_in,), hidden_units, n_actions, actor_critic_together=True)
network.summary()
init_msmt = np.zeros(
    (1, N_in)
)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
hidden_0 (Dense)                (None, 100)          300         input_1[0][0]                    
__________________________________________________________________________________________________
hidden_1 (Dense)                (None, 100)          10100       hidden_0[0][0]                   
__________________________________________________________________________________________________
hidden_2 (Dense)                (None, 100)          10100       hidden_1[0][0]                   
______________________________________________________________________________________________

In [12]:
# Plotting tools
plt.rcParams["figure.dpi"] = 300
plt.rcParams["savefig.dpi"] = 300
avg_return = np.zeros(n_epochs)
fidelities = np.zeros(n_epochs)
visualization_steps = 10
%matplotlib inline

In [ ]:
# Training loop

mu_old = tf.Variable(initial_value=network(init_msmt)[0][0], trainable=False)
sigma_old = tf.Variable(initial_value=network(init_msmt)[1][0], trainable=False)

for i in tqdm(range(n_epochs)):

    Old_distrib = MultivariateNormalDiag(
        loc=mu_old, scale_diag=sigma_old, validate_args=True, allow_nan_stats=False
    )
    obs, _ = q_env.reset()
    with tf.GradientTape(persistent=True) as tape:

        mu, sigma, b = network(np.array([obs]), training=True)
        mu = tf.squeeze(mu, axis=0)
        sigma = tf.squeeze(sigma, axis=0)
        b = tf.squeeze(b, axis=0)
        q_env.mean_action = np.array(mu)
        q_env.std_action = np.array(sigma)
        Policy_distrib = MultivariateNormalDiag(
            loc=mu, scale_diag=sigma, validate_args=True, allow_nan_stats=False
        )

        action_vector = tf.stop_gradient(
            tf.clip_by_value(Policy_distrib.sample(batch_size), -1.0, 1.0)
        )

        _, reward, _, _, _ = q_env.step(action_vector)
        advantage = reward - b

        if use_PPO:
            ratio = Policy_distrib.prob(action_vector) / (
                tf.stop_gradient(Old_distrib.prob(action_vector)) + 1e-7
            )
            actor_loss = -tf.reduce_mean(
                tf.minimum(
                    advantage * ratio,
                    advantage * tf.clip_by_value(ratio, 1 - epsilon, 1 + epsilon),
                )
            )
        else:
            actor_loss = -tf.reduce_mean(advantage * Policy_distrib.log_prob(action_vector))

        critic_loss = tf.reduce_mean(advantage**2)
        combined_loss = actor_loss + critic_loss_coeff * critic_loss

    grads = tape.gradient(combined_loss, network.trainable_variables)

    if use_PPO:
        mu_old.assign(mu)
        sigma_old.assign(sigma)
    avg_return[i] = np.mean(q_env.reward_history[-1])
    if q_env.do_benchmark():
        fidelities[i] = q_env.fidelity_history[-1]
    print("Fidelity", fidelities[i])
    if i % visualization_steps == 0:
        clear_output(wait=True)
        fig, ax = plt.subplots()
        ax.plot(
            np.arange(1, i, 1),
            avg_return[0:i-1],
            "-.",
            label="Avg return",
        )
        ax.plot(
            np.arange(1, i, 1),
            fidelities[0:i-1],
            label="State Fidelity",
        )
        ax.set_xlabel("Epoch")
        ax.set_ylabel("State Fidelity")
        ax.legend()
        plt.show()
        print("Maximum fidelity reached so far:", np.max(fidelities))

    optimizer.apply_gradients(zip(grads, network.trainable_variables))
q_env.close()

In [ ]:
print("Maximum fidelity reached:", np.max(fidelities), "at Epoch ", np.argmax(fidelities))

In [ ]:
plt.plot(np.mean(q_env.reward_history, axis=1), label="Average return")
plt.plot(q_env.circuit_fidelity_history, label="State fidelity")
plt.xlabel("Epoch")
plt.ylabel("Average return")